In [1]:
import pandas as pd
import numpy as np
from preprocess import preprocess

In [2]:
sales = pd.read_csv('data/caspecoTrainingData.csv')
sales = preprocess(sales)
sales

,Date,Company,Sales,Day,Week,Month,Year,Weekday,Weekend,Workday,Holiday,Holieve,Payweek,Payday,Closed
2,2020-01-01,0,0.000000,1,1,1,0,2,0,1,1,0,0,0,1
1,2020-01-01,1,0.000000,1,1,1,0,2,0,1,1,0,0,0,1
0,2020-01-01,2,72608.623649,1,1,1,0,2,0,1,1,0,0,0,0
5,2020-01-02,0,25421.830339,2,1,1,0,3,0,1,0,0,0,0,0
4,2020-01-02,1,57957.790779,2,1,1,0,3,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,2023-01-03,1,65686.415376,3,1,1,3,1,0,1,0,0,0,0,0
3294,2023-01-03,2,411655.230277,3,1,1,3,1,0,1,0,0,0,0,0
3299,2023-01-04,0,36221.562895,4,1,1,3,2,0,1,0,0,0,0,0
3298,2023-01-04,1,60815.193532,4,1,1,3,2,0,1,0,0,0,0,0
